In [1]:
import os
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import col, when
import shutil

# Caminho real do seu Python 3.11
PYTHON_PATH = r"C:\Users\Moises Drummer\anaconda3\python.exe"

# Define o Python correto para Spark
os.environ["PYSPARK_PYTHON"] = PYTHON_PATH
os.environ["PYSPARK_DRIVER_PYTHON"] = PYTHON_PATH

In [2]:
file_path = "breweries.json"
spark = SparkSession.builder \
        .master("local[1]") \
        .config("spark.driver.memory", "1g") \
        .config("spark.local.dir", "C:/temp/spark-temp") \
        .appName("Processed_layer") \
        .getOrCreate()
# Lê o JSON em um DataFrame
silver_df = pd.read_json(file_path)
spark_df = spark.createDataFrame(silver_df)
# Exibe as 5 primeiras linhas
print(silver_df.head())

                                     id                     name brewery_type  \
0  5128df48-79fc-4f0f-8b52-d06be54d0cec         (405) Brewing Co        micro   
1  9c5a66c8-cc13-416f-a5d9-0a769c87d318         (512) Brewing Co        micro   
2  34e8c68b-6146-453f-a4b9-1f6cd99a5ada  1 of Us Brewing Company        micro   
3  6d14b220-8926-4521-8d19-b98a2d6ec3db     10 Barrel Brewing Co        large   
4  e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2     10 Barrel Brewing Co        large   

                     address_1 address_2 address_3            city  \
0               1716 Topeka St      None      None          Norman   
1        407 Radam Ln Ste F200      None      None          Austin   
2          8100 Washington Ave      None      None  Mount Pleasant   
3                62970 18th St      None      None            Bend   
4  1135 NW Galveston Ave Ste B      None      None            Bend   

  state_province postal_code        country   longitude   latitude  \
0       Oklahoma  7306

In [12]:
    silver_df = spark_df.select(
        col("id"),
        col("name"),
        col("brewery_type"),
        col("street"),
        col("city"),
        col("state"),
        col("postal_code"),
        col("country"),
        col("longitude").cast("double").alias("longitude"),
        col("latitude").cast("double").alias("latitude"),
        col("phone"),
        col("website_url")
    ).withColumn("state", when(col("state").isNull(), "UNKNOWN").otherwise(col("state")))


In [15]:
# Converter para Pandas e escrever
silver_output_path = "file:///C:/Users/Moises Drummer/repo/bess_case - Copia/arquivos_saida"
spark_pd = silver_df.toPandas()
silver_pd.to_parquet(silver_output_path.replace("file:///", ""), partition_cols=["state"])

In [13]:
silver_output_path = "file:///C:/Users/Moises Drummer/repo/bess_case - Copia/arquivos_saida"

if os.path.exists(silver_output_path):
    shutil.rmtree(silver_output_path)
    
silver_df.write \
    .mode("overwrite") \
    .partitionBy("state") \
    .parquet(silver_output_path)

Py4JJavaError: An error occurred while calling o185.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (DESKTOP-NFTL0RP executor driver): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to file:/C:/Users/Moises Drummer/repo/bess_case - Copia/arquivos_saida.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:775)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:420)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:100)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:893)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:893)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.

In [ ]:
spark.sparkContext.parallelize([0]).map(lambda x: __import__("sys").version).collect()
